In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torchvision
import torchvision.transforms as T
import torch


tensor([[4.1308e-02, 3.0003e-01, 1.3335e-01],
        [3.2140e-01, 9.9629e-01, 1.7011e-01],
        [6.6198e-01, 1.8704e-04, 6.9423e-02],
        [5.4556e-01, 7.2055e-01, 6.9178e-01],
        [7.3549e-01, 1.6445e-02, 4.1641e-01]])


In [2]:
def detect_person(image_path):
    # Cargar el modelo preentrenado de MobileNet-SSD
    prototxt = "../modelos_preentrenados/deploy.prototxt"
    model = "../modelos_preentrenados/mobilenet_iter_73000.caffemodel"
    net = cv2.dnn.readNetFromCaffe(prototxt, model)

    # Cargar la imagen y obtener sus dimensiones
    image = cv2.imread(image_path)
    (h, w) = image.shape[:2]

    # Preprocesar la imagen
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

    # Pasar la imagen preprocesada a través de la red
    net.setInput(blob)
    detections = net.forward()

    person_detected = False

    # Analizar las detecciones
    for i in range(detections.shape[2]):
        # Obtener la confianza (probabilidad) asociada a la detección
        confidence = detections[0, 0, i, 2]

        # Filtrar detecciones con confianza mayor a un umbral (por ejemplo, 0.5)
        if confidence > 0.75:
            # Obtener el índice de la clase de la detección
            idx = int(detections[0, 0, i, 1])

            # Si la clase es "persona" (clase 15 en COCO dataset)
            if idx == 15:
                person_detected = True
                break

    return person_detected

# Ejemplo de uso
image_path = "../images/imagenpreubaK655-1.jpg"
if detect_person(image_path):
    print("Persona detectada en la imagen")
else:
    print("No se detectaron personas en la imagen")

No se detectaron personas en la imagen


In [3]:
cont = 1
while cont <= 11:
    df = pd.read_csv(f'../data/datos{cont}_cleanRGB.csv')
    
    for i in df.ph_nombre:
        imagen = '../images/'+i
        if detect_person(imagen):
            df.loc[df.ph_nombre == i, 'ph_persona'] = 1

    df.to_csv(f'../data/datos{cont}_cleanRGBCV.csv', index=False)
    cont += 1

In [2]:

def detect_animals(image_path):
    # Cargar el modelo preentrenado de YOLOv5s
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s.pt')

    # Cargar la imagen
    img = Image.open(image_path)

    # Realizar la detección de objetos
    results = model(img)

    # Clases de animales en el dataset COCO
    animal_classes = [14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

    animal_detected = False

    # Analizar las detecciones
    for det in results.pred[0]:
        class_id = int(det[5])
        confidence = det[4]

        # Filtrar detecciones con confianza mayor a un umbral (por ejemplo, 0.5)
        if confidence > 0.75:
            # Si la clase es un animal
            if class_id in animal_classes:
                animal_detected = True
                break

    return animal_detected



In [4]:
cont = 1
while cont <= 11:
    df = pd.read_csv(f'../data/datos{cont}_cleanRGB.csv')
    
    for i in df.ph_nombre:
        imagen = '../images/'+i
        if detect_animals(imagen):
            df.loc[df.ph_nombre == i, 'ph_animal'] = 1

    df.to_csv(f'../data/datos{cont}_cleanRGBCV2.csv', index=False)
    cont += 1

/Users/ruben/opt/anaconda3/lib/python3.9/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /Users/ruben/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-3-16 Python-3.9.13 torch-2.0.0 CPU



[Errno 2] No such file or directory: '/Users/ruben/opt/anaconda3/lib/python3.9/site-packages/torchvision-0.13.1a0.dist-info/METADATA'


100%|██████████| 14.1M/14.1M [00:04<00:00, 3.69MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /Users/ruben/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-16 Python-3.9.13 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /Users/ruben/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-16 Python-3.9.13 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /Users/ruben/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-16 Python-3.9.13 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /Users/ruben/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-16 Python-3.9.13 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213

In [ ]:

def is_nature(image_path):

    # Cargar el modelo preentrenado de resnet50
    model = torchvision.models.resnet50(pretrained=True)
    model.eval()

    # Clases de elementos naturales en ImageNet (árboles, montañas, ríos, etc.)
    nature_classes = [388, 589, 561, 562, 563, 564, 565, 566, 567]

    # Preprocesar la imagen
    preprocess = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    img = Image.open(image_path)
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)

    # Pasar la imagen preprocesada a través de la red
    with torch.no_grad():
        output = model(input_batch)

    # Calcular la probabilidad de las clases de elementos naturales
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    nature_probabilities = probabilities[nature_classes].sum()

    # Verificar si la probabilidad de elementos naturales supera el umbral
    return nature_probabilities >= 0.75

In [ ]:
cont = 1
while cont <= 11:
    if cont == 8:
        cont += 1
        pass
    else:
        df = pd.read_csv(f'../data/datos{cont}_cleanRGB.csv')
        
        for i in df.ph_nombre:
            imagen = '../images/'+i
            if is_nature(imagen):
                df.loc[df.ph_nombre == i, 'ph_naturaleza'] = 1

        df.to_csv(f'../data/datos{cont}_cleanRGBCV3.csv', index=False)
        cont += 1

In [2]:
def image_entropy(image_path):
    image = cv2.imread(image_path)
    if len(image.shape) == 2:
        print("BYN")
        # Imagen en escala de grises
        hist = cv2.calcHist([image], [0], None, [256], [0, 256])
        max_entropy = np.log2(256)
    else:
        # Imagen a color
        hist_r = cv2.calcHist([image], [0], None, [256], [0, 256])
        hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
        hist_b = cv2.calcHist([image], [2], None, [256], [0, 256])
        hist = hist_r + hist_g + hist_b
        max_entropy = np.log2(256**3)

    hist /= hist.sum()
    entropy = -np.sum(hist * np.log2(hist + np.finfo(float).eps))
    normalized_entropy = entropy / max_entropy

    return normalized_entropy

def image_energy(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist /= hist.sum()
    energy = np.sum(hist**2)
    return energy

def image_focus(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    focus = np.var(laplacian)
    return focus

In [ ]:
cont = 1
while cont <= 11:
    if cont == 8:
        cont += 1
        pass
    else:
        df = pd.read_csv(f'../data/datos{cont}_cleanRGB.csv')
        df["ph_entropia"] = 0
        df["ph_energia"] = 0
        df["ph_foco"] = 0
        for i in df.ph_nombre:
            imagen = '../images/'+i
            df.loc[df.ph_nombre == i, 'ph_entropia'] = image_entropy(imagen)
            df.loc[df.ph_nombre == i, 'ph_energia'] = image_energy(imagen)
            df.loc[df.ph_nombre == i, 'ph_foco'] = image_focus(imagen)
        
        df.to_csv(f'../data/datos{cont}_cleanRGBCV4.csv', index=False)
        cont += 1

In [5]:
imagen = "../images/imagenpreuba569-1.jpg"
print(image_entropy(imagen))


0.223151961962382
